In [43]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score as auc, accuracy_score
import numpy as np

In [44]:
df

,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,ground_name,team_1_hue
0,20,2023,West Ham United,Brighton & Hove Albion,London Stadium,2
1,20,2023,Liverpool,Newcastle United,Anfield,1
2,20,2023,Fulham,Arsenal,Craven Cottage,1
3,20,2023,Tottenham Hotspur,Bournemouth,Tottenham Hotspur Stadium,1
4,20,2023,Nottingham Forest,Manchester United,The City Ground,1
...,...,...,...,...,...,...
1713,1,2019,Burnley,Southampton,Turf Moor,1
1714,1,2019,Crystal Palace,Everton,Selhurst Park,2
1715,1,2019,Watford,Brighton & Hove Albion,Vicarage Road,0
1716,1,2019,West Ham United,Manchester City,London Stadium,0


In [56]:
df = pd.read_csv('../prepare data/data/base_line.csv')
df.head()

,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,ground_name,team_1_hue
0,20,2023,West Ham United,Brighton & Hove Albion,London Stadium,2
1,20,2023,Liverpool,Newcastle United,Anfield,1
2,20,2023,Fulham,Arsenal,Craven Cottage,1
3,20,2023,Tottenham Hotspur,Bournemouth,Tottenham Hotspur Stadium,1
4,20,2023,Nottingham Forest,Manchester United,The City Ground,1


In [57]:
train = df.iloc[50:, :]
val = df.iloc[:50, :]

In [60]:
y_train = train['team_1_hue']
x_train = train.drop('team_1_hue', axis=1)

y_val = val['team_1_hue']
x_val = val.drop('team_1_hue', axis=1)

In [61]:
model_baseline = CatBoostClassifier()

In [62]:
model_baseline.fit(x_train, y_train, cat_features=list(x_train.columns), verbose=False)

In [63]:
pred = model_baseline.predict(x_val)
pred_proba = model_baseline.predict_proba(x_val)
y_val = y_val.to_numpy().reshape(-1, 1)

In [64]:
pred = np.concatenate((y_val, pred, pred_proba), axis=1)
df_pred = pd.DataFrame(pred, columns=['true', 'pred', '0_prob', '1_prob', '2_prob'])

In [65]:
df_pred['error'] = np.where(df_pred['true']==df_pred['pred'], 0, 1)
df_pred.head()


,true,pred,0_prob,1_prob,2_prob,error
0,2.0,1.0,0.257377,0.526374,0.216249,1
1,1.0,1.0,0.043313,0.850491,0.106196,0
2,1.0,0.0,0.627190,0.095494,0.277317,1
3,1.0,1.0,0.067463,0.877842,0.054695,0
4,1.0,0.0,0.522341,0.192453,0.285206,1


In [66]:
df_pred['error'].sum()

22

In [67]:
df_pred[df_pred['2_prob']>0.5]

,true,pred,0_prob,1_prob,2_prob,error
13,0.0,2.0,0.273880,0.225946,0.500175,1
42,1.0,2.0,0.184109,0.177472,0.638419,1


In [68]:
accuracy_score(df_pred['true'], df_pred['pred'])

0.56